### A pyiron workflow to calculate a free surface energy using multiple potentials, and compare to literature DFT data.

This notebook __doesn't__ work out of the box if you haven't configured LAMMPS for pyiron

conda install -c conda-forge lammps 

OR

mamba install -c conda-forge lammps

In the python environment in which you are running pyiron

WARNING: In practice/production you should configure a compiled LAMMPS executable that is optimised for your cluster!

In [1]:
from pyiron.project import Project

import numpy as np
import pandas as pd

from ase.lattice.cubic import BodyCenteredCubic as bcc
from ase.build import stack

from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure

from pyiron.atomistics.structure.atoms import ase_to_pyiron, pymatgen_to_pyiron

In [2]:
pr = Project("FS_calcs")

Create the structure that we're going to use:
a $(111)$ Fe free surface (slab)

In [3]:
def add_vacuum(structure, vacuum):
    """
    Adds padding to a slab or 2D material.

    Args:
        structure (Structure): Structure to add vacuum to
        vacuum (float): Vacuum thickness to add in Angstroms
    Returns:
        Structure object with vacuum added.
    """
    struct = structure.copy()
    coords = [s.coords for s in struct.sites]
    species = [s.specie for s in struct.sites]
    lattice = struct.lattice.matrix
    lattice.setflags(write=1)
    lattice_C = lattice
    lattice_C[2][2] += vacuum
    struct = Structure(lattice_C, species, coords, coords_are_cartesian=True)
    
    return struct

In [4]:
surface1 = [1, 1, 1]
rotation_axis = [1, -1, 0]
element = "Fe"
lc = 2.8318488966083
GB_name = "S3-RA110-S1-11"

# The minimum required length of the cell.
req_length = 15

v1 = list(-np.cross(rotation_axis,surface1))

length = 0
n = 0
while length < req_length:
    n += 1
    
    slab1 = bcc(symbol=element, latticeconstant=lc,directions=[rotation_axis,v1,surface1], size=[1,1,n])

    slab = stack(slab1, slab1)
    length = slab.cell[-1,-1]
    slab.rattle()
    
    pmg_slab = AseAtomsAdaptor().get_structure(slab.copy())
    
    pmg_slab_vac = pmg_slab.copy()
    pmg_slab_vac = add_vacuum(pmg_slab_vac, 15)

Create the pyiron job, assign the structure to the job.

We need to see the potentials, pyiron has an inbuilt potential database

It automatically finds the applicable potentials given the structure provided.

In [5]:
job = pr.create_job(job_type=pr.job_type.Lammps, job_name="test")
job.structure = pymatgen_to_pyiron(pmg_slab)
job.list_potentials()[:10]

['1997--Ackland-G-J--Fe--LAMMPS--ipr1',
 '1998--Meyer-R--Fe--LAMMPS--ipr1',
 '2001--Lee-B-J--Fe--LAMMPS--ipr1',
 '2001--Lee-B-J--Fe-Cr--LAMMPS--ipr1',
 '2003--Mendelev-M-I--Fe-2--LAMMPS--ipr3',
 '2003--Mendelev-M-I--Fe-5--LAMMPS--ipr1',
 '2004--Ackland-G-J--Fe-P--LAMMPS--ipr1',
 '2004--Zhou-X-W--Fe--LAMMPS--ipr2',
 '2005--Lee-B-J--Fe-Cu--LAMMPS--ipr1',
 '2005--Mendelev-M-I--Al-Fe--LAMMPS--ipr1']

In [6]:
potentials_tested = job.list_potentials()
FS_energy_list = []
for i, potential in enumerate(potentials_tested):
    try:
        job_bulk_name = f"bulk_slab_111_{i}"
        job_bulk = pr.create_job(job_type=pr.job_type.Lammps, job_name=job_bulk_name)
        job_bulk.structure = pymatgen_to_pyiron(pmg_slab)
        job_bulk.potential = potential
        job_bulk.calc_minimize(pressure=None,
                            max_iter=10000)
        job_bulk.run(delete_existing_job=True)
        
        job_SLAB_name = f"FS_slab_111_{i}"
        job_SLAB = pr.create_job(job_type=pr.job_type.Lammps, job_name=job_SLAB_name)
        job_SLAB.structure = pymatgen_to_pyiron(pmg_slab_vac)
        job_SLAB.potential = potential
        job_SLAB.calc_minimize(pressure=None,
                            max_iter=10000)
        job_SLAB.run(delete_existing_job=True)
        
        # Formula for GB energy is E_GB_tot - E_SLAB_tot / 2 * (Interface area)
        # The 2 in the denominator is to account for the second interface present in the cell due to the periodic conditions
        
        FS_energy = (job_bulk['output/generic/energy_tot'][-1] - job_SLAB['output/generic/energy_tot'][-1]) \
                    / (2 * job_SLAB.structure.cell.volume / job_SLAB.structure.cell[-1,-1]) * 16.02
        
        FS_energy_list.append(FS_energy)
    except:
        FS_energy_list.append(np.nan)

2023-08-11 18:23:47,719 - pyiron_log - WARNING - No HDF5 file found - remove database entry and create new job! bulk_slab_111_0


The job bulk_slab_111_0 was saved and received the ID: 765


2023-08-11 18:23:48,408 - pyiron_log - WARNING - No HDF5 file found - remove database entry and create new job! FS_slab_111_0


The job FS_slab_111_0 was saved and received the ID: 766


2023-08-11 18:23:48,993 - pyiron_log - WARNING - No HDF5 file found - remove database entry and create new job! bulk_slab_111_1


...

The job bulk_slab_111_122 was saved and received the ID: 1124
The job bulk_slab_111_123 was saved and received the ID: 1125


2023-08-11 18:26:36,893 - pyiron_log - WARNING - Job aborted
2023-08-11 18:26:36,895 - pyiron_log - WARNING - LAMMPS (28 Mar 2023 - Update 1)
Reading data file ...
  orthogonal box = (0 0 0) to (4.0048391 6.9365848 34.619625)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  48 atoms
  read_data CPU = 0.000 seconds
ERROR: KIM ModelCreate failed (src/KIM/pair_kim.cpp:828)
Last command: pair_style kim Sim_LAMMPS_MEAM_LiyanageKimHouze_2014_FeC__SM_652425777808_000

2023-08-11 18:26:36,906 - pyiron_log - WARNING - No HDF5 file found - remove database entry and create new job! bulk_slab_111_124
2023-08-11 18:26:37,075 - pyiron_log - WARNING - Job aborted
2023-08-11 18:26:37,076 - pyiron_log - WARNING - LAMMPS (28 Mar 2023 - Update 1)
Reading data file ...
  orthogonal box = (0 0 0) to (4.0048391 6.9365848 34.619625)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  48 atoms
  read_data CPU = 0.000 seconds
ERROR: KIM ModelCreate failed (src/KIM/pair_kim.cpp:828)
Last command: pair_

The job bulk_slab_111_124 was saved and received the ID: 1126
The job bulk_slab_111_125 was saved and received the ID: 1127


2023-08-11 18:26:37,262 - pyiron_log - WARNING - Job aborted
2023-08-11 18:26:37,263 - pyiron_log - WARNING - LAMMPS (28 Mar 2023 - Update 1)
Reading data file ...
  orthogonal box = (0 0 0) to (4.0048391 6.9365848 34.619625)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  48 atoms
  read_data CPU = 0.000 seconds
ERROR: KIM ModelCreate failed (src/KIM/pair_kim.cpp:828)
Last command: pair_style kim Sim_LAMMPS_TersoffZBL_ByggmastarGranberg_2020_Fe__SM_958863895234_000

2023-08-11 18:26:37,279 - pyiron_log - WARNING - No HDF5 file found - remove database entry and create new job! bulk_slab_111_126
2023-08-11 18:26:37,454 - pyiron_log - WARNING - Job aborted
2023-08-11 18:26:37,455 - pyiron_log - WARNING - LAMMPS (28 Mar 2023 - Update 1)
Reading data file ...
  orthogonal box = (0 0 0) to (4.0048391 6.9365848 34.619625)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  48 atoms
  read_data CPU = 0.000 seconds
ERROR: KIM ModelCreate failed (src/KIM/pair_kim.cpp:828)
Last command

The job bulk_slab_111_126 was saved and received the ID: 1128


Construct a dataframe which contains the information about FS energy, and their errors with respect to a DFT computed value.

The FS energy of 2.73 J/m^2 is from the value computed in this study:

Mai, H.L., Cui, X.Y., Scheiber, D., Romaner, L. and Ringer, S.P., 2022. The segregation of transition metals to iron grain boundaries and their effects on cohesion. Acta materialia, 231, p.117902.

In [7]:
data = {'potential': potentials_tested, 'FS_energy (J/m^2)': FS_energy_list}
df = pd.DataFrame(data)

FS_energy_DFT = 2.73
df["err_DFT (J/m^2)"] = np.round(FS_energy_DFT - df["FS_energy (J/m^2)"], 3)
df["abs_rel_err_DFT (%)"] = abs(np.round((FS_energy_DFT - df["FS_energy (J/m^2)"]) / FS_energy_DFT * 100, 0))
df["FS_energy (J/m^2)"] = np.round(df["FS_energy (J/m^2)"].tolist(), 3)
df = df.sort_values("abs_rel_err_DFT (%)")
df = df.dropna(subset="FS_energy (J/m^2)")
df

,potential,FS_energy (J/m^2),err_DFT (J/m^2),abs_rel_err_DFT (%)
66,2022--Starikov-S--Fe-Cr-H--LAMMPS--ipr1,2.786,-0.056,2.0
21,2009--Kim-H-K--Fe-Ti-C--LAMMPS--ipr2,2.648,0.082,3.0
56,2019--Mendelev-M-I--Fe-Ni-Cr--LAMMPS--ipr1,2.355,0.375,14.0
27,2010--Malerba-L--Fe--LAMMPS--ipr1,2.337,0.393,14.0
35,2012--Proville-L--Fe--LAMMPS--ipr1,3.182,-0.452,17.0
...,...,...,...,...
0,1997--Ackland-G-J--Fe--LAMMPS--ipr1,16.566,-13.836,507.0
55,2019--Byggmastar-J--Fe-O--LAMMPS--ipr1,18.084,-15.354,562.0
40,2013--Henriksson-K-O-E--Fe-C--LAMMPS--ipr1,18.084,-15.354,562.0
32,2011--Chiesa-S--Fe-33--LAMMPS--ipr1,18.308,-15.578,571.0


Order the potentials by the minimum error wrt. DFT!

Now we have a good idea of how to search for an empirical potential which best captures the FS energetics of a specific surface.

You can see that there are only a few potentials that are very, very accurate at predicting this specific surface energy. 

For a real study you should consider not just a single surface, but validate it across multiple surfaces and ideally the specific phenomena you want to study.

In [8]:
df.sort_values(by="abs_rel_err_DFT (%)").head(10)

,potential,FS_energy (J/m^2),err_DFT (J/m^2),abs_rel_err_DFT (%)
66,2022--Starikov-S--Fe-Cr-H--LAMMPS--ipr1,2.786,-0.056,2.0
21,2009--Kim-H-K--Fe-Ti-C--LAMMPS--ipr2,2.648,0.082,3.0
56,2019--Mendelev-M-I--Fe-Ni-Cr--LAMMPS--ipr1,2.355,0.375,14.0
27,2010--Malerba-L--Fe--LAMMPS--ipr1,2.337,0.393,14.0
35,2012--Proville-L--Fe--LAMMPS--ipr1,3.182,-0.452,17.0
1,1998--Meyer-R--Fe--LAMMPS--ipr1,2.101,0.629,23.0
33,2012--Jelinek-B--Al-Si-Mg-Cu-Fe--LAMMPS--ipr2,2.075,0.655,24.0
59,2020--Groger-R--Co-Cr-Fe-Mn-Ni--LAMMPS--ipr1,2.026,0.704,26.0
7,2004--Zhou-X-W--Fe--LAMMPS--ipr2,3.639,-0.909,33.0
50,2018--Farkas-D--Fe-Ni-Cr-Co-Cu--LAMMPS--ipr2,1.593,1.137,42.0
